<h1>Técnicas básicas de Procesamiento de Lenguaje Natural</h1>


<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03%20Machine%20Learning/notebooks/10-NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En esta notebook aprenderemos algunas técnicas de NLP para lidiar con las tareas de aprendizaje supervisado en el contexto del texto escrito.

En el NLP hay dos librerias clásicas principales:

* [NLTK](https://www.nltk.org/)
* [Spacy](https://spacy.io/)

Revisaremos:
* Preprocesamiento y limpieza de texto
* Tecnicas de representación de texto
* Algoritmos especializados: Naive Bayes Multinomial, la métrica angular

Además, lidiaremos con un problema típico del aprendizaje supervisado: el desbalanceo de clases.

El problema que abordaremos será construir un modelo que identifique mensajes SPAM. Es un problema de clasificación binaria. En este contexto, queremos minimizar los falsos positivos, es decir, queremos evitar que marque mensajes no spam como spam. La métrica que monitorea los falsos positivos especificamente es el precision, recordar que

$$\text{Precision} = \frac{TP}{TP+FP}$$

Usaremos Accuracy y Precision para este problema, principalmente. Sin embargo, al tratarse de un problema desbalanceado hacia la clase negativa, es importante ver el recall o el F1-score. Muchas de las predicciones exitosas pueden deberse a que se esta apoyando en la clase mayoritaria.

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/03%20Machine%20Learning/data/Spam_SMS.csv "

df = pd.read_csv(url, encoding='utf-8')
df

## Análisis Exploratorio

En el NLP es importante hacer análisis exploratorio de los datos y determinar parámetros y caracteristicas importantes, por ejemplo:

* Idioma del corpus
* Distribución de la longitud de los documentos
* Perfilado de texto: Análisis técnico que examina estructura, metadatos, uso de caracteres/emojis y anomalías
* Fuente del corpus
* Calidad del corpus

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

longitudes = [len(x.split()) for x in df['Message'].values]
longitud_promedio = np.mean(longitudes)

plt.figure()
plt.hist(longitudes)
plt.xlabel('Longitud')
plt.ylabel('Frecuencia')
plt.axvline(x=longitud_promedio, color='r', linestyle='--', label=f'Longitud promedio: {longitud_promedio:.1f}')
plt.legend()
plt.title('Distribución de longitudes de mensajes')
plt.show()

Nos apoyamos de herramientas como *wordclouds* para ver palabras comunes y patrones de vocabulario

In [ ]:
!pip install -qq wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

texto = " ".join(df['Message'])

plt.figure(figsize=(10, 10))
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(texto)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

🔵 ¿Qué conclusiones podemos sacar de este gráfico?

Las **stopwords** son palabras frecuentes pero con bajo valor semántico ("el", "y", "de", "en", etc.).

* Es recomendable eliminarlas en tareas de análisis de texto (como clasificación, minería de datos o SEO) para reducir ruido y enfocarse en términos clave.
* No es recomendable elimnarlas cuando se necesita preservar estructura gramatical (en chatbots, traducción automática o análisis sintáctico), ya que su eliminación puede distorsionar el significado.

Mientras en modelos de machine learning clásico suelen omitirse para eficiencia, en generación de lenguaje natural o modelos más modernos y costosos son esenciales.

Además, su relevancia y listado varía por idioma y dominio (por ejemplo, en textos legales, *artículo* podría ser stopword).

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt_tab')

stopwords = nltk.corpus.stopwords.words('english')  # También podemos escoger las stopwords en español

Veamos algunas stopwords:

In [ ]:
print(stopwords[:10])

Definimos una función para limpieza de texto.


La **tokenización** es el proceso de dividir un texto en unidades mínimas (tokens), como palabras, símbolos o frases, para su análisis en NLP.

    Texto: "¡Hola, mundo!" → Tokens: ["¡", Hola", ",", "mundo", "!"].

In [ ]:
from nltk import word_tokenize

def clean_text(text):
    text = text.lower()
    text = word_tokenize(text)
    text = [word for word in text if word not in stopwords]
    text = " ".join(text)
    return text

df['Clean Message'] = df['Message'].apply(clean_text)
df

Ahora hagamos una nube de palabras con el texto limpio

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

texto_limpio = " ".join(df['Clean Message'])

plt.figure(figsize=(10, 10))
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(texto_limpio)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

Hagamos una nube de palabras por cada etiqueta, con los textos limpios

In [ ]:
textos_limpios_ham = " ".join(df[df['Class'] == 'ham']['Clean Message'])
textos_limpios_spam = " ".join(df[df['Class'] == 'spam']['Clean Message'])

plt.figure(figsize=(15, 10))
plt.subplot(1, 2, 1)
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(textos_limpios_ham)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Ham')
plt.subplot(1, 2, 2)
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(textos_limpios_spam)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Spam')
plt.show()

## Preprocesamiento

Hay técnicas adicionales muy usadas para procesar el texto:

- **Expresiones regulares (RegEx)**  
  *Definición:* Patrones de texto utilizados para buscar, coincidir y manipular cadenas de caracteres.  
  *Uso:* Validación de formatos (ej. emails), extracción de información, sustitución de texto y filtrado de datos.  

|          |                                                                  |
|------------------------|----------------------------------------------------------------------------------------|
| **Texto inicial**      | `Contacta a soporte@empresa.com o a ventas@tienda.com. Para errores, escribe a bugs@dev.org. No envíes spam a info@dominio.invalido.` |
| **Expresión regular**  | `\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b`                                  |
| **Coincidencias**      | 1. `soporte@empresa.com`<br>2. `ventas@tienda.com`<br>3. `bugs@dev.org`

- **Lematizar**  
  *Definición:* Proceso lingüístico que reduce una palabra a su forma base o canónica (lema). Ej: "corriendo" → "correr".  
  *Uso:* Normalización de texto en PLN (Procesamiento de Lenguaje Natural) para mejorar análisis de frecuencia o búsquedas semánticas.  

- **POS Tagging (Etiquetado gramatical)**  
  *Definición:* Asignación de categorías gramaticales (sustantivo, verbo, adjetivo, etc.) a cada palabra en un texto.  
  *Uso:* Análisis sintáctico, traducción automática, generación de texto y sistemas de chatbots para entender la estructura de frases.  


No veremos estos puntos.

Separación de variables

In [ ]:
y = df['Class'].values
# texts = df['Message'].values
texts = df['Clean Message'].values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ratio = y[y == 'spam'].shape[0] / y.shape[0]
plt.suptitle(f'Distribución de clases\n{ratio:.2%} spam')
sns.countplot(y)
plt.show()

Como podemos ver, es un dataset moderadamente desbalanceado. No hay umbrales precisos. Una guía empírica es:

* Balanceado: La clase minoritaria representa > 30% del total.
* Desbalanceado moderado: Clase minoritaria entre 10% y 30%.
* Extremadamente desbalanceado: Clase minoritaria < 10%.
* Si la clase minoritaria tiene < 5% de los datos, se considera un problema severo (requiere técnicas especiales como oversampling/SMOTE o cost-sensitive learning).

Codificación de clases

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, y_train, y_test = train_test_split(texts,
                                                            y,
                                                            test_size=0.2,
                                                            stratify=y,
                                                            random_state=1942)

In [ ]:
from sklearn.preprocessing import LabelEncoder

print(y_train[:5])

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

print(y_train[:5])


## Extracción de features

Hay dos métodos clásicos para vectorizar texto. Ambos funcionan bajo el mismo principio: convertir documentos de texto en vectores numéricos basados en la frecuencia de las palabras, pero difieren en cómo ponderan estos términos.

* [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).  

 * Representa cada documento como un vector donde cada componente cuenta la frecuencia absoluta de una palabra en el texto.
 * No considera la importancia relativa de las palabras en el corpus, solo su ocurrencia.
* [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

 * Ajusta la frecuencia de las palabras ponderando su importancia en el documento y en el corpus. La importancia está dada por el producto de dos términos:

   1. TF (Term Frequency): Frecuencia del término en el documento.
   2. IDF (Inverse Document Frequency): Penaliza términos comunes (como "el", "y") y da más peso a palabras relevantes.

`CountVectorizer` solo cuenta palabras, mientras que `TfidfVectorizer` ajusta los pesos para reflejar qué tan único o relevante es un término en el corpus.

🔽 Ejemplo Ilustrativo

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ["el gato come pescado", "el perro come pan"]
for j,doc in enumerate(corpus):
    print(f"documento {j+1}: {doc}")

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.toarray())  # Matriz de conteo
print(vectorizer.get_feature_names_out())  # Vocabulario

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ["el gato come pescado", "el perro come pan"]
for j,doc in enumerate(corpus):
    print(f"documento {j+1}: {doc}")

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.toarray().round(3))  # Matriz de pesos
print(vectorizer.get_feature_names_out())  # Vocabulario

## Entrenamiento y Evaluación

Regresemos a nuestro ejemplo

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
                            stop_words='english',
                            max_features=None,
                            lowercase=True
                            )
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [ ]:
X_train.toarray()[:3,:5]

In [ ]:
print(X_train.shape)
print(f"Hay {X_train.shape[0]} documentos y {X_train.shape[1]} features (palabras)")

Es decir, cada documento está representado por un vector de 7493 componentes. ¡Podemos tener un problema de dimensionalidad!

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

clf = DecisionTreeClassifier(max_depth=10)
clf.fit(X_train, y_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

print(f"Train accuracy: {accuracy_score(y_train, y_pred_train):.2%}")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.2%}")

print(f"Train precision: {precision_score(y_train, y_pred_train):.2%}")
print(f"Test precision: {precision_score(y_test, y_pred_test):.2%}")

print(f"Train recall: {recall_score(y_train, y_pred_train):.2%}")
print(f"Test recall: {recall_score(y_test, y_pred_test):.2%}")

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(30, 30),dpi=300)
plot_tree(clf, filled=True,
          feature_names=vectorizer.get_feature_names_out(),
          class_names=le.classes_,
          impurity=False
          )
plt.savefig('tree.png')
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

print(f"Train accuracy: {accuracy_score(y_train, y_pred_train):.2%}")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.2%}")

print(f"Train precision: {precision_score(y_train, y_pred_train):.2%}")
print(f"Test precision: {precision_score(y_test, y_pred_test):.2%}")

print(f"Train recall: {recall_score(y_train, y_pred_train):.2%}")
print(f"Test recall: {recall_score(y_test, y_pred_test):.2%}")

Veamos un clasificador probabilisto ideal para tareas de clasificación en NLP: **Naive Bayes** Multinomial

<img src="https://drive.google.com/uc?id=164QWAMozr4nyUvU1F05jvEk-9zxP9NmJ" alt="alt text" width="500">

La implementación en scikit learn es [`MultinomialNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html). Su hiperparámetro principal es `alpha` que es una constante de suavizado que se usa al estimar probabilidades en términos de conteos

$$P(w) = \frac{\text{frecuencia} + \alpha}{\text{vocabulario} + \alpha}$$


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, recall_score

clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

print(f'Train accuracy: {accuracy_score(y_train, y_pred_train):.2%}')
print(f'Test accuracy: {accuracy_score(y_test, y_pred_test):.2%}')

print(f"Train precision: {precision_score(y_train, y_pred_train):.2%}")
print(f"Test precision: {precision_score(y_test, y_pred_test):.2%}")

print(f"Train recall: {recall_score(y_train, y_pred_train):.2%}")
print(f"Test recall: {recall_score(y_test, y_pred_test):.2%}")

In [ ]:
cm = confusion_matrix(y_test, y_pred_test)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

Validemos este modelo con *Cross Validation*

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
print(np.mean(scores))

scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='precision')
print(np.mean(scores))

## GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

param_grid = {
    'vectorizer__max_features': [None, 5000, 10000],
    'selector__k': [100, 500, 1000, 5000],
    'classifier__alpha': [0.01, 0.1, 1.0, 10.0]
}

pipeline = Pipeline([
    ('vectorizer', CountVectorizer(lowercase=True, stop_words='english')),
    ('selector', SelectKBest()),
    ('classifier', MultinomialNB())
])

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='precision')
grid_search.fit(train_texts, y_train)

print("Mejores parámetros:", grid_search.best_params_)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score

best_nb_model = grid_search.best_estimator_
y_pred_test = best_nb_model.predict(test_texts)
y_pred_train = best_nb_model.predict(train_texts)

print(f"Train accuracy: {accuracy_score(y_train, y_pred_train):.2%}")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.2%}")

print(f"Train precision: {precision_score(y_train, y_pred_train):.2%}")
print(f"Test precision: {precision_score(y_test, y_pred_test):.2%}")

print(f"Train recall: {recall_score(y_train, y_pred_train):.2%}")
print(f"Test recall: {recall_score(y_test, y_pred_test):.2%}")

cm = confusion_matrix(y_test, y_pred_test)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


Veamos las palabras que mejor contribuyeron a la tarea

In [ ]:
selection_mask = best_nb_model['selector'].get_support()  # mascara de selección
feature_names = best_nb_model['vectorizer'].get_feature_names_out() # palabras del vectorizador
selected_features = feature_names[selection_mask]
selected_features

## Comparación con otros clasificadores

Como podemos ver, la mayoria de las clasificaciones han tenido buen desempeño en el accuracy y precision. Comparemos varios clasificadores usando el recall.

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
import numpy as np
import time
import pandas


clfs = {
    'SVC': SVC(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForestClassifier': RandomForestClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'MultinomialNB': MultinomialNB(),
    'DecisionTreeClassifier': DecisionTreeClassifier()
}

accs = []
times = []

vectorizer = CountVectorizer(lowercase=True, stop_words='english')
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

for name, clf in clfs.items():
    these_times = []
    for i in range(5):
        start = time.time()
        clf.fit(X_train, y_train)
        end = time.time()
        these_times.append(end - start)
    times.append(np.mean(these_times))
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='recall')
    accs.append(np.mean(scores))

results_df = pandas.DataFrame({'clf': list(clfs.keys()), 'recall': accs, 'time': times})

In [ ]:
results_df.sort_values('recall', ascending=False)

In [ ]:
results_df.sort_values('time')

Como podemos ver, el Naive Bayes es un método muy efectivo y barato en tareas de NLP.

## Vectorización TF-IDF

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(lowercase=True, stop_words='english')),
    ('classifier', MultinomialNB())
])

pipeline.fit(train_texts, y_train)
y_pred_test = pipeline.predict(test_texts)
y_pred_train = pipeline.predict(train_texts)

print(f"Train accuracy: {accuracy_score(y_train, y_pred_train):.2%}")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.2%}")

print(f"Train precision: {precision_score(y_train, y_pred_train):.2%}")
print(f"Test precision: {precision_score(y_test, y_pred_test):.2%}")

## ⚡¿Cómo lidiamos con el desbalanceo de clases?

Tres estrategias muy usadas son:

* **RandomUnderSampler**: Reduce la clase mayoritaria eliminando ejemplos al azar hasta equilibrar las clases. Es rápido, pero puede perder información útil.

* **RandomOverSampler**: Aumenta la clase minoritaria copiando ejemplos existentes al azar. Simple, pero puede causar sobreajuste al repetir los mismos datos.

* **SMOTE**: En lugar de copiar datos, crea ejemplos sintéticos de la clase minoritaria combinando muestras similares. Mejora la variedad de los datos, pero a veces genera ejemplos poco realistas.

Por ahora, probemos *undersampling*. En notebooks posteriores probaremos SMOTE.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=1992)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_train_rus shape: {X_train_rus.shape}")

In [ ]:
y_train

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

original_counts, rus_counts = np.unique(y_train, return_counts=True), np.unique(y_train_rus, return_counts=True)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.bar(original_counts[0], original_counts[1])
plt.title('Original')
plt.subplot(1, 2, 2)
plt.bar(rus_counts[0], rus_counts[1])
plt.title('Undersampled')
plt.show()

🔵 ¿Por qué no hacemos lo mismo con el conjunto de prueba?

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

clf = SVC()
clf.fit(X_train_rus, y_train_rus)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

print(f"Train accuracy: {accuracy_score(y_train, y_pred_train):.2%}")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.2%}")

print(f"Train precision: {precision_score(y_train, y_pred_train):.2%}")
print(f"Test precision: {precision_score(y_test, y_pred_test):.2%}")

Como podemos ver, no ayuda mucho al desempeño del modelo. Algunas razones son:

* Tamaño del dataset
* Perdida de información relevante

## 🔽 Representaciones vectoriales de documentos

El hecho de usar `CountVectorizer` o el `TfidfVectorizer` para vectorizar los documentos se puede ver de dos maneras:

1. **Extracción de features**: El texto no posee intrínsecamente features que definan al texto, por lo que estas técnicas transforman las palabras en características numéricas basadas en su frecuencia o importancia.
2. **Representación vectorial** de documentos: Estas herramientas convierten los textos en vectores numéricos, permitiendo su procesamiento matemático por medio de algoritmos de ML. A diferencia de los word embeddings, que capturan significado y contexto, estos métodos se centran en la representación superficial del texto.


<img src="https://drive.google.com/uc?id=1FxlQrTa2Vg7_QGRVlfYIM5z9tSmAzXZW" alt="alt text" width="500">



Exploremos los vectores de documentos

In [ ]:
n_entradas = 80

X_train.toarray()[0,:n_entradas]

Observa que estos vectores son casi puros ceros, es decir, son *sparse*. Tenemos 3 opciones para combatir esto:

1. Seleccionar features.
2. Combinar features.
3. Vectorizar de otra manera (embeddings).

In [ ]:
import numpy as np

idx1 = np.random.choice(X_train.shape[0])
idx2 = np.random.choice(X_train.shape[0])

v1 = X_train.toarray()[idx1,:]
v2 = X_train.toarray()[idx2,:]

distancia = np.linalg.norm(v1 - v2).round(3)
print(f"Distancia entre los vectores de los documentos {idx1} y {idx2}: {distancia}")

similitud = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
print(f"Similitud entre los vectores de los documentos {idx1} y {idx2}: {similitud}")